# **Step1: Install amd import libraries**

In [10]:
!pip install -qq "transformers==4.35" "peft==0.4.0" "accelerate==0.21.0" "bitsandbytes==0.40.2" "trl==0.4.7" "safetensors>=0.3.1" "tiktoken"
!pip install -qq gradio
!pip install -qq gtts
import pandas as pd
import gradio as gr
from gtts import gTTS
import os
import torch
from datasets import Dataset, load_dataset
from random import randrange
from peft import LoraConfig, get_peft_model, AutoPeftModelForCausalLM
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from trl import SFTTrainer
import torch
from peft import AutoPeftModelForCausalLM
import transformers

print(transformers.__version__)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Device ',device,' is being used')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 54.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.10/dist-packages/transformers/utils/import_utils.py:533: FutureWarning: `is_torch_tpu_available` is deprecated and will be removed in 4.41.0. Please use the `is_torch_xla_available` instead.
  


ImportError: cannot import name 'top_k_top_p_filtering' from 'transformers' (/usr/local/lib/python3.10/dist-packages/transformers/__init__.py)

# **Step 2: Load Saved Model**

In [ ]:
# Model_3
! gdown 17Z8n0X53ho-C_vcc1m2bKanf8pwjt_dw
print(' If you cannot download dataset: please use this link:\n https://drive.google.com/file/d/17Z8n0X53ho-C_vcc1m2bKanf8pwjt_dw/view?usp=sharing')


In [ ]:
!unzip -qq '/content/Final_Laama_Model.zip'

In [ ]:
model_directory = '/content/results/lamma/checkpoint-2100' # Set this to your directory

new_model = AutoPeftModelForCausalLM.from_pretrained(
    model_directory,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="auto",
)

##2-1 Load tokenizer

In [ ]:

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_directory, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# **Step 3: Generate answer**

In [ ]:
def answer_query(query,temperature=0.6,long_answer = False):

    if long_answer:
      prompt= 'Generatee long answer for this question ' + query

    else:
      prompt= 'Generate short answer for this question ' + query

    input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()

    if long_answer:

        outputs = new_model.generate(input_ids=input_ids,


                             min_length=100,
                             max_new_tokens=300,
                                 #repetition_penalty=0.2,
                            #  do_sample=True,
                            #  top_p=0.9,
                             temperature=temperature)

    else:

        outputs = new_model.generate(input_ids=input_ids,


                             min_length=40,
                             #max_length = 100,
                             max_new_tokens=70,
                                 #repetition_penalty=0.2,
                            #  do_sample=True,
                            #  top_p=0.9,
                             temperature=temperature)
    answer = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]

    return prompt,answer

## 3-1 Make answer pretty

In [ ]:
# Remove question from answer and remove last uncomplete sentence
def pretty_asnwer(prompt,answer):
    new_answer = answer
    if new_answer.endswith('.') or new_answer.endswith('!') or new_answer.endswith('?'):

        last_index_of_question = len(prompt)
        new_answer = new_answer[last_index_of_question:]
        if new_answer.find('Answer')>-1:
          index = new_answer.find('Answer')
          new_answer = new_answer[index+8:]
        #print(new_answer)

    else:
        #print('Not complete answer')
        last_index_of_question = len(prompt)
        new_answer = new_answer[last_index_of_question:]
        last_sentence = new_answer.split('. ')[-1]
        new_answer = new_answer.replace(last_sentence, " ")
        if new_answer.find('Answer')>-1:
          #print('here')
          index = new_answer.find('Answer')
          new_answer = new_answer[index+8:]



    return new_answer



# **Step 4: Ueser Interface**

In [ ]:

# Check availabilty:
def disability(query,long_answer):
  result = 0
  while len(result)<20: # Generate again if, last time model was disable
      answer = answer_query(query,long_answer=long_answer)
      new_answer = pretty_asnwer(answer)
      result = new_answer

  return result

def text_to_speech(text):
        # Convert text to speech
        tts = gTTS(text)
        # Save the converted audio to a file
        tts.save("output.mp3")
        # Return the file path
        return "output.mp3"

def aks_question(Question, Long_Format, Short_Format, Audio_Play ):
    query = Question

    #check Long format

    if Long_Format == True and Short_Format == True:
        # query_copy_1 = query
        # query_copy_2 = query

        prompt,long_answer = answer_query(query,temperature=0.6,long_answer = True)
        long_answer = pretty_asnwer(prompt,long_answer)

        while len(long_answer)<50: # Generate again if, last time model was disable
            prompt,long_answer = answer_query(query,temperature=0.6,long_answer = True)
            long_answer = pretty_asnwer(long_answer)


        prompt,short_answer = answer_query(query,temperature=0.6,long_answer = False)
        short_answer = pretty_asnwer(prompt,short_answer)


        while len(short_answer)<30: # Generate again if, last time model was disable
          prompt,short_answer = answer_query(query,temperature=0.6,long_answer = False)
          short_answer = pretty_asnwer(prompt,short_answer)

        if Audio_Play:
          text = 'Long answer:'+ long_answer + 'Short answer:'+short_answer
          audio_ouput = text_to_speech(text)
          return long_answer, short_answer, audio_ouput

        else:
          text = ' You did not chose audio play'
          audio_ouput = text_to_speech(text)
          return long_answer, short_answer, audio_ouput


    elif Long_Format == True and Short_Format == False:
        prompt,long_answer = answer_query(query,temperature=0.6,long_answer = True)
        long_answer = pretty_asnwer(prompt,long_answer)

        while len(long_answer)<50: # Generate again if, last time model was disable
            prompt,long_answer = answer_query(query,temperature=0.6,long_answer = True)
            long_answer = pretty_asnwer(long_answer)


        if Audio_Play:
          text = 'Long answer:'+ long_answer
          audio_ouput = text_to_speech(text)
          return long_answer, ' ', audio_ouput

        else:
          text = ' You did not chose audio play'
          audio_ouput = text_to_speech(text)
          return long_answer, ' ', audio_ouput


    elif Long_Format == False and Short_Format == True:
        prompt,short_answer = answer_query(query,temperature=0.6,long_answer = False)
        short_answer = pretty_asnwer(prompt,short_answer)

        while len(short_answer)<30: # Generate again if, last time model was disable
          prompt,short_answer = answer_query(query,temperature=0.6,long_answer = False)
          short_answer = pretty_asnwer(prompt,short_answer)

        if Audio_Play:
          text = 'short answer:'+ short_answer
          audio_ouput = text_to_speech(text)
          return ' ',short_answer , audio_ouput

        else:
          text = ' You did not chose audio play'
          audio_ouput = text_to_speech(text)
          return ' ', short_answer, audio_ouput


    else:
        text = ' Please check one checkbox at least'
        audio_ouput = text_to_speech(text)
        return 'Please check one checkbox at least', 'Please check one checkbox at least', audio_ouput


demo = gr.Interface(
    fn=aks_question,

    inputs=["text", "checkbox","checkbox","checkbox"],

    outputs=[
        gr.Textbox(label="Long Answer"),
        gr.Textbox(label="Short Answer"),
        gr.Audio(label='Long_Format_Audio_Play')
    ]
)
demo.launch()

In [1]:
!gdown 15GwtRsgdPNkgQ-15WLsmZa6Dsd-hijmH


Downloading...
From (original): https://drive.google.com/uc?id=15GwtRsgdPNkgQ-15WLsmZa6Dsd-hijmH
From (redirected): https://drive.google.com/uc?id=15GwtRsgdPNkgQ-15WLsmZa6Dsd-hijmH&confirm=t&uuid=27bd57ba-b535-4c18-a29f-0b5ae404cb2d
To: /content/checkpoint_epoch_1_batch_40000.pt
100% 2.68G/2.68G [00:41<00:00, 65.0MB/s]


In [9]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, AdamW, get_linear_schedule_with_warmup
import random
import os
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Training parameters
epochs = 2
batch_size = 1
learning_rate = 5e-5
adam_epsilon = 1e-8
warmup_steps = 0
train_questions = 91772


# Load the T5 tokenizer and model
model_name = 't5-base'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

model.to(device)

# Optimizer and scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, eps=adam_epsilon)
total_steps = (train_questions) // batch_size * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps)

def load_checkpoint(checkpoint_path, model, optimizer, scheduler):
    checkpoint = torch.load(checkpoint_path,  map_location=torch.device(device))
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    start_epoch = checkpoint['epoch'] + 1  # start from the next epoch
    loss = checkpoint['loss']
    return model, optimizer, scheduler, start_epoch, loss

# Initialize variables
start_epoch = 0  # default start epoch
checkpoint_path = '/content/checkpoint_epoch_1_batch_40000.pt'

model, optimizer, scheduler, start_epoch, loss = load_checkpoint(checkpoint_path, model, optimizer, scheduler)
print(f"Loaded checkpoint from epoch {start_epoch}, loss: {loss}")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded checkpoint from epoch 2, loss: 75456.31543746311


In [ ]:
def generate_answer(question, context):
    input_text = f"question: {question} context: {' '.join(context[0] + context[1])}"
    inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=1024, truncation=True).to(device)
    outputs = model.generate(inputs, min_length=100, max_length=1024, num_beams=5, early_stopping=True).to(device)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

cntx = find_relevant_context_SBERT(question, question_embeddings, all_answers, all_scores, faiss_index, use='test')
answer = generate_answer(question, cntx)
print(answer)